# HLT Assignment

In addition to practising the process of loading data, regularizing it and using it to train a model, the goal of this assignment is to investigat the effect of changing model parameters on the performance.

## Step 1: loading the data

We'll be going a bit further with the robot collision dataset. This time, instead of looking at just the first file, we'll look at all five different tasks combined (lp1.data to lp5.data). Prepare two different arrays, X1 and X2, as follows:
- Each element in X1 is the immediate reading of the force and torque values after an event, [f1, f2, f3, t1, t2, t3]. The first element should be [1, 1, 63, 3, 1, 0]
- Each element in X2 contains 18 values in total - the first, fifth and tenth sets of sensor readings after an event. The first element should be [-1, -1, 61, -3, 0, 0, -1, -1, 63, -3, -1, 0, -1, -1, 61, -3, 0, 0]

y should contain the corresponding classes, represented as integers according the the provided dictionary

In [1]:
files = ['robot_execution_failure/lp1.data', 'robot_execution_failure/lp2.data',
'robot_execution_failure/lp3.data', 'robot_execution_failure/lp4.data', 'robot_execution_failure/lp5.data']
classes = {'normal':0, 'collision':1, 'obstruction':2, 'fr_collision':3}

In [2]:
# inputs
X1 = [] 
X2 = []

# true values
y = []

for i in range(len(files) - 1):
    filename = files[i]
    f = open(filename)
    lines = f.readlines() # Read the file line by line into a list
    for i in range(len(lines) - 1):
        line = lines[i].strip() # .strip() removes the line endings \n
        if line in classes.keys(): # If the line matches one of our classes (for eg, 'normal')
            featuresX1 = [int(x) for x in lines[i+1].strip().split('\t')]
            temp = lines[i+1].strip().split('\t') + lines[i+5].strip().split('\t')+ lines[i+10].strip().split('\t')
            featuresX2 = [int(x) for x in temp]
            X1.append(featuresX1)
            X2.append(featuresX2)
            y.append(classes[line]) # And record which class this set of features belongs to

In [3]:
X1[0]

[-1, -1, 63, -3, -1, 0]

In [4]:
X2[0]

[-1, -1, 63, -3, -1, 0, -1, -1, 63, -3, -1, 0, -1, -1, 61, -3, 0, 0]

In [5]:
y[0]

0

In [6]:
# Take note that the first entry of X1 and X2 do not match the suggested values in the task. 
#        After checking the actual files, It was determined that the values obtained in X1[0] and X2[0] 
#        above are correct as per the actual data file contents.

## Step 2: establishing a baseline

Using techniques covered in this unit, split X1 and y into separate training and testing sets. Use the training set to train a neural network (MLPClassifier) using default parameters but with hidden_layer_sizes=(20, 20, 20). Use the test data you held back to score the model you have created. How well does it perform? Print out the score and confusion matrix. For more accuracy, run through these steps 10 times and find the average score - bonus points for running more times and getting a standard deviation!

In [7]:
# Creating a training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y)

In [8]:
# Preparing to scale the inputs
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [9]:
# Scaling the inputs 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# Creating the neural network
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))

In [11]:
# Training
mlp.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 20, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [12]:
# Predict and print score
predictions = mlp.predict(X_test)
print (mlp.score(X_test, y_test))

0.7017543859649122


If you're getting a convergence warning, you could try having the model train over more iterations - change max_iter = 1000 or 10,000. Does this improve the average score?

In [13]:
# Set max iterations to 1000
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20), max_iter = 1000)
mlp.fit(X_train, y_train)
print (mlp.score(X_test, y_test))

0.7368421052631579


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [14]:
# Set max iterations to 10000
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20), max_iter = 10000)
mlp.fit(X_train, y_train)
print (mlp.score(X_test, y_test))

0.6491228070175439


In [15]:
# Initialise scores array
scores = []

# Repeat 20 times
count = 0
while count < 20:
    mlp.fit(X_train, y_train) # Training
    mlp.score(X_test, y_test)
    scores.append(mlp.score(X_test, y_test))
    count +=1
    
# Print the Standard Deviation of the scores
import numpy as np
print('Standard Deviation:')
print(np.std(scores))

predictions = mlp.predict(X_test)

# Print the Confusion Matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,predictions))

Standard Deviation:
0.041692051362512784
[[17  3  0  0]
 [ 1 13  4  0]
 [ 1  4  8  0]
 [ 2  0  1  3]]


## Step 3: adding more inputs

Use X2 in place of X1 - does the score increase or decrease? Was this what you expected? How many samples are there in our training set?

In [16]:
len(X2)

225

In [17]:
# Repeating the steps above with X2 instead of X1
X_train, X_test, y_train, y_test = train_test_split(X2, y)
scaler = StandardScaler()
scaler.fit(X_train)
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))
mlp.fit(X_train, y_train)
print (mlp.score(X_test, y_test))

0.7192982456140351


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [18]:
# The score is better with the added inputs. You would expect this since it has more dimensions to describe a condition.

## Step 4: feature engineering

Back to X1 as our input. Add an extra feature to each item in the array to represent the total force $f_t$. Assume:

$f_t^2 = f_1^2 + f_2^2 + f_3^2$

Your first input should now look like this:
X1[0] = [-1, -1, 61, -3, 0, 0, 61.0163912403872]

Repeat the steps from step 2. *Has this extra feature improved model performance?*

In [19]:
# inputs
X1 = [] 

# true values
y = []

for i in range(len(files) - 1):
    filename = files[i]
    f = open(filename)
    lines = f.readlines() # Read the file line by line into a list
    for i in range(len(lines) - 1):
        line = lines[i].strip() # .strip() removes the line endings \n
        if line in classes.keys(): # If the line matches one of our classes (for eg, 'normal')
            featuresX1 = [int(x) for x in lines[i+1].strip().split('\t')]
            X1.append(featuresX1)
            y.append(classes[line]) # And record which class this set of features belongs to

In [20]:
# Creating the new feature and appending it to the dataframe
for i in range(len(X1)):
    totalForce = (((X1[i][0])**2)+((X1[i][1])**2)+((X1[i][2])**2))**(0.5)
    X1[i].append(totalForce)

In [21]:
X1[0]

[-1, -1, 63, -3, -1, 0, 63.0158710167526]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X1, y)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20), max_iter = 1000)
mlp.fit(X_train, y_train)
print (mlp.score(X_test, y_test))

0.7543859649122807


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [23]:
# It is observed that the new feature decreased the score.

## Step 5: exploring

Create at least 3 more models, adding features or changing the number and size of the hidden layers. Print out the average score for your best model. Comment on what you've found.

In [24]:
# inputs
X1 = [] 

# true values
y = []

for i in range(len(files) - 1):
    filename = files[i]
    f = open(filename)
    lines = f.readlines() # Read the file line by line into a list
    for i in range(len(lines) - 1):
        line = lines[i].strip() # .strip() removes the line endings \n
        if line in classes.keys(): # If the line matches one of our classes (for eg, 'normal')
            featuresX1 = [int(x) for x in lines[i+1].strip().split('\t')]
            X1.append(featuresX1)
            y.append(classes[line]) # And record which class this set of features belongs to

In [25]:
# Model 1 (More hidden layers)
X_train, X_test, y_train, y_test = train_test_split(X1, y)
scaler = StandardScaler() # Preparing to scale the inputs
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPClassifier(hidden_layer_sizes=(90,90,90))
mlp.fit(X_train, y_train)
print(mlp.score(X_test, y_test))

0.8421052631578947


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [26]:
# Model 2 (More less layers)
mlp = MLPClassifier(hidden_layer_sizes=(5,5,5))
mlp.fit(X_train, y_train)
print(mlp.score(X_test, y_test))

0.22807017543859648


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [27]:
# Adding more features by adding the 1st, 4th, 8th, 12th and 15th sets of sensor readings after an event
# (Excluding the total force)

X1 = [] # inputs
y  = [] # true values

for i in range(len(lines) - 1):
    line = lines[i].strip() # .strip() removes the line endings \n
    if line in classes.keys(): # If the line matches one of our classes (for eg, 'normal')
        features = [int(x) for x in lines[i+1].strip().split('\t')] # Split the next line to get our features
        X1lines = lines[i+1] + lines[i+4] + lines[i+8] + lines[i+12] + lines[i+15]
        features2 = [int(x) for x in X1lines.strip().split('\t')] # Split the next line to get our features
        X1.append(features2)
        y.append(classes[line]) # And record which class this set of features belongs to        

X_train, X_test, y_train, y_test = train_test_split(X1, y)
scaler = StandardScaler() # Preparing to scale the inputs
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))
mlp.fit(X_train, y_train)
print(mlp.score(X_test, y_test))

0.8


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
